In [1]:
import requests,os
import pandas as pd
import numpy as np

In [2]:
#Pyspark library
import findspark
findspark.init()

import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import col, split, when, regexp_replace, trim,explode,arrays_zip

In [3]:
# Initialize Spark
def initial_spark():
    # Create Spark Session
    sc = SparkContext()
    spark = SparkSession(sc).builder.appName("Example").getOrCreate()
    print(f"The current spark version = { spark.version }.")
    return spark

In [4]:
#Initialiase spark and make iteritems in pandas into items for faster processing
spark = initial_spark()
pd.DataFrame.iteritems = pd.DataFrame.items

The current spark version = 3.5.1.


In [5]:
#import the file
exercise_data = pd.read_excel('psdexercise.xlsx')

#Explore some rows of data
exercise_data.head()

,ปี,ภาค,จังหวัด,อำเภอ,ตำบล,เพศ,อายุ,การศึกษา,สถานภาพ,การมีโรคประจำตัว,...,นาที/วัน,วัน/สัปดาห์,รวมสรุปกิจกรรม,สถานที่,เวลา,เวลารวม (ชั่วโมง),เหตุผล (ออก),เหตุผล (ไม่ออก),แรงจูงใจ,ข้อเสนอแนะ
0,2564,ภาคกลาง,สมุทรปราการ,เมืองสมุทรปราการ,สำโรงเหนือ,หญิง,55,มัธยมศึกษาตอนต้น,แยกกันอยู่,ไม่มี,...,(1)45,(1)5,"(ระดับปานกลาง,225)","บ้าน/บริเวณที่พักอาศัย ,สวนสาธารณะ ,ถนน/ทางสาธ...","(06.00 - 07.00 น.),(18.00 - 19.00 น.)",2,"ต้องการให้ร่างกายแข็งแรง,ต้องการรูปร่างที่ดี/ห...",-,ความรู้ในการออกกำลังกาย,ปรับปรุงซ่อมแซม สวนสาธารณะที่มีอยู่เดิมให้มีสภ...
1,2564,ภาคกลาง,สมุทรปราการ,เมืองสมุทรปราการ,สำโรงเหนือ,หญิง,47,มัธยมศึกษาตอนปลาย สายอาชีพ,สมรส,ไม่มี,...,-,-,-,-,-,-,-,"ไม่มีเวลา,สถานการณ์โควิด-19",การจัดกิจกรรมที่น่าสนใจ,จัดให้มีบุคลากรทางการกีฬามาให้ความรู้ในการออกก...
2,2564,ภาคกลาง,สมุทรปราการ,เมืองสมุทรปราการ,สำโรงเหนือ,ชาย,45,ปริญญาตรี,สมรส,ไม่มี,...,-,-,-,-,-,-,-,"ไม่มีเวลา,ไม่มีสถานที่",สนามกีฬาหรือสถานที่ที่มีความสวยงามมีมาตรฐาน,สร้างสวนสาธารณะเพิ่มขึ้น
3,2564,ภาคกลาง,สมุทรปราการ,เมืองสมุทรปราการ,สำโรงเหนือ,ชาย,53,มัธยมศึกษาตอนปลาย สายสามัญ,สมรส,ไม่มี,...,(1)45,(1)3,"(ระดับปานกลาง,135)","บ้าน/บริเวณที่พักอาศัย ,ลานออกกำลังกาย/เล่นกีฬ...","(06.00 - 07.00 น.),(18.00 - 19.00 น.)",2,"ต้องการให้ร่างกายแข็งแรง,ควบคุมน้ำหนัก/ลดน้ำหน...",-,ทัศนคติในการออกกำลังกาย,ปรับปรุงซ่อมแซม สวนสาธารณะที่มีอยู่เดิมให้มีสภ...
4,2564,ภาคกลาง,สมุทรปราการ,เมืองสมุทรปราการ,สำโรงเหนือ,ชาย,57,ประถมศึกษา,สมรส,"เบาหวาน,ความดันโลหิตสูง",...,(1)60,(1)3,"(ระดับปานกลาง,180)",ลานออกกำลังกาย/เล่นกีฬา,"(07.00 - 08.00 น.),(18.00 - 19.00 น.)",2,"ต้องการให้ร่างกายแข็งแรง,รักษา/บรรเทาอาการเจ็บ...",-,ผู้นำการออกกำลังกาย,สนับสนุนอุปกรณ์การกีฬา เครื่อง ออกกำลังกายก...


In [6]:
exercise_data[exercise_data.columns[10:16]].head()

,ประเภทของโรค,อาชีพ,น้ำหนัก,ส่วนสูง,กิจกรรม_2564,ระดับ
0,ไม่มี,รับจ้างทั่วไป,53,155,(1)ปั่นจักรยาน,(1)ระดับปานกลาง
1,ไม่มี,ประกอบธุรกิจส่วนตัว,56,167,-,-
2,ไม่มี,รับจ้างทั่วไป,73,166,-,-
3,ไม่มี,พนักงานบริษัท/ห้างร้าน,65,160,(1)วิ่ง,(1)ระดับปานกลาง
4,โรคเบาหวานและความผิดปกติของต่อมไร้ท่อและเมตะบอ...,รับราชการ/เจ้าหน้าที่ของรัฐ,130,165,(1)ปั่นจักรยาน,(1)ระดับปานกลาง


In [7]:
# Get the information of this dataset
print("Get information on this dataset.")
print(exercise_data.info())

Get information on this dataset.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20098 entries, 0 to 20097
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   ปี                 20098 non-null  int64 
 1   ภาค                20098 non-null  object
 2   จังหวัด            20098 non-null  object
 3   อำเภอ              20098 non-null  object
 4   ตำบล               20098 non-null  object
 5   เพศ                20098 non-null  object
 6   อายุ               20098 non-null  object
 7   การศึกษา           20098 non-null  object
 8   สถานภาพ            20098 non-null  object
 9   การมีโรคประจำตัว   20098 non-null  object
 10  ประเภทของโรค       20098 non-null  object
 11  อาชีพ              20098 non-null  object
 12  น้ำหนัก            20098 non-null  object
 13  ส่วนสูง            20098 non-null  object
 14  กิจกรรม_2564       20098 non-null  object
 15  ระดับ              20098 non-null  object
 16  นาที/วั

In [8]:
exercise_data[exercise_data['แรงจูงใจ'].isnull() == True]

,ปี,ภาค,จังหวัด,อำเภอ,ตำบล,เพศ,อายุ,การศึกษา,สถานภาพ,การมีโรคประจำตัว,...,นาที/วัน,วัน/สัปดาห์,รวมสรุปกิจกรรม,สถานที่,เวลา,เวลารวม (ชั่วโมง),เหตุผล (ออก),เหตุผล (ไม่ออก),แรงจูงใจ,ข้อเสนอแนะ
12929,2565,กรุงเทพมหานคร,กรุงเทพมหานคร,หนองจอก,กระทุ่มราย,ชาย,25,ปวส./ปวท./อนุปริญญา,โสด,โรคกระเพาะ,...,(1)30,(1)3,"(ระดับปานกลาง,90)",บ้าน/บริเวณที่พักอาศัย,(07.00 - 08.00 น.),1,"ต้องการให้ร่างกายแข็งแรง,ควบคุมน้ำหนัก/ลดน้ำหนัก",-,NaN,อุปกรณ์การกีฬาควรมีราคาถูกลง


In [9]:
exercise_data = exercise_data.fillna('ไม่ระบุ')
exercise_data= exercise_data.reset_index()

In [10]:
fields = [StructField(field_name, StringType(), True) for field_name in exercise_data.columns]
schema = StructType(fields)

exercise_df = spark.createDataFrame(exercise_data,schema=schema)
exercise_df.show(5)

+-----+----+-------+-----------+----------------+----------+----+----+--------------------+----------+--------------------+--------------------+--------------------+-------+-------+--------------+---------------+--------+-----------+------------------+--------------------+--------------------+-----------------+--------------------+--------------------+--------------------+--------------------+
|index|  ปี|    ภาค|    จังหวัด|           อำเภอ|      ตำบล| เพศ|อายุ|            การศึกษา|   สถานภาพ|    การมีโรคประจำตัว|        ประเภทของโรค|               อาชีพ|น้ำหนัก|ส่วนสูง|  กิจกรรม_2564|          ระดับ|นาที/วัน|วัน/สัปดาห์|    รวมสรุปกิจกรรม|             สถานที่|                เวลา|เวลารวม (ชั่วโมง)|        เหตุผล (ออก)|     เหตุผล (ไม่ออก)|            แรงจูงใจ|          ข้อเสนอแนะ|
+-----+----+-------+-----------+----------------+----------+----+----+--------------------+----------+--------------------+--------------------+--------------------+-------+-------+--------------+----------

### Data Cleaning

In [11]:
exercise_df = exercise_df.withColumnRenamed('เวลา','ช่วงเวลาออกกำลังกาย').withColumnRenamed('กิจกรรม_2564','กิจกรรม').withColumnRenamed('index','ID')
exercise_df.select(['ช่วงเวลาออกกำลังกาย','กิจกรรม']).show(5)

+--------------------+--------------+
| ช่วงเวลาออกกำลังกาย|       กิจกรรม|
+--------------------+--------------+
|(06.00 - 07.00 น....|(1)ปั่นจักรยาน|
|                   -|             -|
|                   -|             -|
|(06.00 - 07.00 น....|       (1)วิ่ง|
|(07.00 - 08.00 น....|(1)ปั่นจักรยาน|
+--------------------+--------------+
only showing top 5 rows



In [12]:
exercise_df = exercise_df.withColumn('กิจกรรม',regexp_replace('กิจกรรม',"\(\d+\)","")).withColumn('ระดับ',regexp_replace('ระดับ',"\(\d+\)","")).withColumn('นาที/วัน',regexp_replace('นาที/วัน',"\(\d+\)","")).withColumn('วัน/สัปดาห์',regexp_replace('วัน/สัปดาห์',"\(\d+\)",""))
exercise_df.show(5)

<>:1: SyntaxWarning: invalid escape sequence '\('
<>:1: SyntaxWarning: invalid escape sequence '\('
<>:1: SyntaxWarning: invalid escape sequence '\('
<>:1: SyntaxWarning: invalid escape sequence '\('
<>:1: SyntaxWarning: invalid escape sequence '\('
<>:1: SyntaxWarning: invalid escape sequence '\('
<>:1: SyntaxWarning: invalid escape sequence '\('
<>:1: SyntaxWarning: invalid escape sequence '\('
C:\Users\USER\AppData\Local\Temp\ipykernel_17244\3318414278.py:1: SyntaxWarning: invalid escape sequence '\('
  exercise_df = exercise_df.withColumn('กิจกรรม',regexp_replace('กิจกรรม',"\(\d+\)","")).withColumn('ระดับ',regexp_replace('ระดับ',"\(\d+\)","")).withColumn('นาที/วัน',regexp_replace('นาที/วัน',"\(\d+\)","")).withColumn('วัน/สัปดาห์',regexp_replace('วัน/สัปดาห์',"\(\d+\)",""))
C:\Users\USER\AppData\Local\Temp\ipykernel_17244\3318414278.py:1: SyntaxWarning: invalid escape sequence '\('
  exercise_df = exercise_df.withColumn('กิจกรรม',regexp_replace('กิจกรรม',"\(\d+\)","")).withColumn('ร

+---+----+-------+-----------+----------------+----------+----+----+--------------------+----------+--------------------+--------------------+--------------------+-------+-------+-----------+------------+--------+-----------+------------------+--------------------+--------------------+-----------------+--------------------+--------------------+--------------------+--------------------+
| ID|  ปี|    ภาค|    จังหวัด|           อำเภอ|      ตำบล| เพศ|อายุ|            การศึกษา|   สถานภาพ|    การมีโรคประจำตัว|        ประเภทของโรค|               อาชีพ|น้ำหนัก|ส่วนสูง|    กิจกรรม|       ระดับ|นาที/วัน|วัน/สัปดาห์|    รวมสรุปกิจกรรม|             สถานที่| ช่วงเวลาออกกำลังกาย|เวลารวม (ชั่วโมง)|        เหตุผล (ออก)|     เหตุผล (ไม่ออก)|            แรงจูงใจ|          ข้อเสนอแนะ|
+---+----+-------+-----------+----------------+----------+----+----+--------------------+----------+--------------------+--------------------+--------------------+-------+-------+-----------+------------+--------+---------

In [13]:
exercise_df = exercise_df.withColumn('กิจกรรม',split(col('กิจกรรม'),',')).withColumn('ระดับ',split(col('ระดับ'),',')).withColumn('นาที/วัน',split(col('นาที/วัน'),',')).withColumn('วัน/สัปดาห์',split(col('วัน/สัปดาห์'),','))
exercise_df.select('กิจกรรม','ระดับ','นาที/วัน','วัน/สัปดาห์').show(5)

+-------------+--------------+--------+-----------+
|      กิจกรรม|         ระดับ|นาที/วัน|วัน/สัปดาห์|
+-------------+--------------+--------+-----------+
|[ปั่นจักรยาน]|[ระดับปานกลาง]|    [45]|        [5]|
|          [-]|           [-]|     [-]|        [-]|
|          [-]|           [-]|     [-]|        [-]|
|       [วิ่ง]|[ระดับปานกลาง]|    [45]|        [3]|
|[ปั่นจักรยาน]|[ระดับปานกลาง]|    [60]|        [3]|
+-------------+--------------+--------+-----------+
only showing top 5 rows



In [14]:
exercise_df= exercise_df.withColumn("new",arrays_zip('กิจกรรม','ระดับ','นาที/วัน','วัน/สัปดาห์'))\
    .withColumn("new",explode('new')).withColumn('กิจกรรม',col('new.กิจกรรม'))\
    .withColumn('ระดับ',col('new.ระดับ')).withColumn('นาที/วัน',col("new.นาที/วัน"))\
    .withColumn('วัน/สัปดาห์',col('new.วัน/สัปดาห์'))
exercise_df.show(5)

+---+----+-------+-----------+----------------+----------+----+----+--------------------+----------+--------------------+--------------------+--------------------+-------+-------+-----------+------------+--------+-----------+------------------+--------------------+--------------------+-----------------+--------------------+--------------------+--------------------+--------------------+--------------------+
| ID|  ปี|    ภาค|    จังหวัด|           อำเภอ|      ตำบล| เพศ|อายุ|            การศึกษา|   สถานภาพ|    การมีโรคประจำตัว|        ประเภทของโรค|               อาชีพ|น้ำหนัก|ส่วนสูง|    กิจกรรม|       ระดับ|นาที/วัน|วัน/สัปดาห์|    รวมสรุปกิจกรรม|             สถานที่| ช่วงเวลาออกกำลังกาย|เวลารวม (ชั่วโมง)|        เหตุผล (ออก)|     เหตุผล (ไม่ออก)|            แรงจูงใจ|          ข้อเสนอแนะ|                 new|
+---+----+-------+-----------+----------------+----------+----+----+--------------------+----------+--------------------+--------------------+--------------------+-------+-------+-

In [15]:
exercise_df = exercise_df.withColumn('เหตุผล (ไม่ออก)',regexp_replace('เหตุผล (ไม่ออก)','-','ไม่ระบุเหตุผล')).drop('new')
exercise_df.show(10)

+---+----+-------+-----------+----------------+----------+----+----+--------------------+----------+--------------------+--------------------+--------------------+-------+-------+--------------------+------------+--------+-----------+------------------+--------------------+--------------------+-----------------+--------------------+--------------------+--------------------+--------------------+
| ID|  ปี|    ภาค|    จังหวัด|           อำเภอ|      ตำบล| เพศ|อายุ|            การศึกษา|   สถานภาพ|    การมีโรคประจำตัว|        ประเภทของโรค|               อาชีพ|น้ำหนัก|ส่วนสูง|             กิจกรรม|       ระดับ|นาที/วัน|วัน/สัปดาห์|    รวมสรุปกิจกรรม|             สถานที่| ช่วงเวลาออกกำลังกาย|เวลารวม (ชั่วโมง)|        เหตุผล (ออก)|     เหตุผล (ไม่ออก)|            แรงจูงใจ|          ข้อเสนอแนะ|
+---+----+-------+-----------+----------------+----------+----+----+--------------------+----------+--------------------+--------------------+--------------------+-------+-------+--------------------+----

In [16]:
for col in exercise_df.columns:
    exercise_df = exercise_df.withColumn(col,when(exercise_df[col]=='-','ไม่ระบุ เพราะไม่ออกกำลังกาย').otherwise(exercise_df[col]))
exercise_df.show(5)

+---+----+-------+-----------+----------------+----------+----+----+--------------------+----------+--------------------+--------------------+--------------------+-------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
| ID|  ปี|    ภาค|    จังหวัด|           อำเภอ|      ตำบล| เพศ|อายุ|            การศึกษา|   สถานภาพ|    การมีโรคประจำตัว|        ประเภทของโรค|               อาชีพ|น้ำหนัก|ส่วนสูง|             กิจกรรม|               ระดับ|            นาที/วัน|         วัน/สัปดาห์|      รวมสรุปกิจกรรม|             สถานที่| ช่วงเวลาออกกำลังกาย|   เวลารวม (ชั่วโมง)|        เหตุผล (ออก)|     เหตุผล (ไม่ออก)|            แรงจูงใจ|          ข้อเสนอแนะ|
+---+----+-------+-----------+----------------+----------+----+----+--------------------+----------+--------------------+---------------

In [17]:
exercise_df = exercise_df.withColumn("อายุ",exercise_df["อายุ"].cast(IntegerType())).withColumn("น้ำหนัก",exercise_df["น้ำหนัก"].cast(IntegerType()))\
    .withColumn("ส่วนสูง",exercise_df["ส่วนสูง"].cast(IntegerType())).withColumn("นาที/วัน",exercise_df["นาที/วัน"].cast(IntegerType()))\
    .withColumn("วัน/สัปดาห์",exercise_df["วัน/สัปดาห์"].cast(IntegerType()))\
    .withColumn("เวลารวม (ชั่วโมง)",exercise_df["เวลารวม (ชั่วโมง)"].cast(IntegerType()))


In [18]:
exercise_df.printSchema()

root
 |-- ID: string (nullable = true)
 |-- ปี: string (nullable = true)
 |-- ภาค: string (nullable = true)
 |-- จังหวัด: string (nullable = true)
 |-- อำเภอ: string (nullable = true)
 |-- ตำบล: string (nullable = true)
 |-- เพศ: string (nullable = true)
 |-- อายุ: integer (nullable = true)
 |-- การศึกษา: string (nullable = true)
 |-- สถานภาพ: string (nullable = true)
 |-- การมีโรคประจำตัว: string (nullable = true)
 |-- ประเภทของโรค: string (nullable = true)
 |-- อาชีพ: string (nullable = true)
 |-- น้ำหนัก: integer (nullable = true)
 |-- ส่วนสูง: integer (nullable = true)
 |-- กิจกรรม: string (nullable = true)
 |-- ระดับ: string (nullable = true)
 |-- นาที/วัน: integer (nullable = true)
 |-- วัน/สัปดาห์: integer (nullable = true)
 |-- รวมสรุปกิจกรรม: string (nullable = true)
 |-- สถานที่: string (nullable = true)
 |-- ช่วงเวลาออกกำลังกาย: string (nullable = true)
 |-- เวลารวม (ชั่วโมง): integer (nullable = true)
 |-- เหตุผล (ออก): string (nullable = true)
 |-- เหตุผล (ไม่ออก): string 